Evaluation & Clustering


# Evaluation

## Q1

A classifier's performance can be captured in a confusion matrix, which in the case of a binary classifier is a two-by-two matrix where the main diagonal has the counts of the "correct" prediction, while the off-diagonal values are the counts of when the prediction was wrong.

Write a function confusion_matrix(classifier, dataset) that takes a classifier and a test dataset, and returns a confusion matrix capturing how well the classifier classified the dataset. The confusion matrix object must be created by calling ConfusionMatrix(tp, fn, fp, tn). The definition of the class is given below. Make sure the definition is included with your answer.

In [ ]:
from collections import namedtuple

class ConfusionMatrix(namedtuple("ConfusionMatrix",
                                 "true_positive false_negative "
                                 "false_positive true_negative")):

    def __str__(self):
        elements = [self.true_positive, self.false_negative,
                   self.false_positive, self.true_negative]
        return ("{:>{width}} " * 2 + "\n" + "{:>{width}} " * 2).format(
                    *elements, width=max(len(str(e)) for e in elements))
        
def confusion_matrix(classifier, dataset):
  results = {'tp' : 0, 'fp' : 0, 'tn' : 0, 'fn' : 0,}
  for point, expected in dataset:
    result = classifier(point)
    if result == expected and result:
        results['tp'] += 1
    elif result == expected and not result:
        results['tn'] += 1
    else:
      if result:
        results['fp'] += 1
      else:
        results['fn'] += 1
  return ConfusionMatrix(results['tp'], results['fn'], results['fp'], results['tn'])
                    

Test


2 0

1 0



2 0

0 1

In [ ]:
dataset = [
    ((0.8, 0.2), 1),
    ((0.4, 0.3), 1),
    ((0.1, 0.35), 0),
]
print(confusion_matrix(lambda x: 1, dataset))
print()
print(confusion_matrix(lambda x: 1 if x[0] + x[1] > 0.5 else 0, dataset))

2 0 
1 0 

2 0 
0 1 


## Q2

When solving a classification problem, we often have many different options for classifiers with different strengths and weaknesses. The domination relation in the ROC space determines which classifiers are outperformed by other classifiers for all possible misclassification costs. In this question you have to write a function that filters out all such outperformed classifiers.

Write a function roc_non_dominated(classifiers) which takes a collection of classifiers and returns only those classifiers that are not dominated by any other classifier in the collection. A classifier is represented as a pair (classifier_name, confusion_matrix), where classifier_name is a string, and confusion_matrix is a named tuple representing the two-by-two classification confusion matrix. The implementation of a confusion matrix is:

from collections import namedtuple

class ConfusionMatrix(namedtuple("ConfusionMatrix",
                                 "true_positive false_negative "
                                 "false_positive true_negative")):
    pass
This is the same as in the previous question (less the __str__ method, which we do not need here). Please include it in the answer.

In addition to the visible test cases, there are several test cases that load classifier data from files. Each file has up to 10000 classifiers. Two sample files are provided below.

roc_small.data
roc.data
You can use the following code to read the files:

classifiers = []
with open(filename) as file:
    for line in f.readlines():
        label, tp, fn, fp, tn = line.strip().split(",")
        classifiers.append((label,
                            ConfusionMatrix(int(tp), int(fn),
                                            int(fp), int(tn))))

In [ ]:
from collections import namedtuple

class ConfusionMatrix(namedtuple("ConfusionMatrix",
                                 "true_positive false_negative "
                                 "false_positive true_negative")):
    pass
        

def roc_non_dominated(classifiers):
    """
    Takes a collection of classifiers and returns only those classifiers that are not dominated by any other classifier in the collection. 
    A classifier is represented as a pair (classifier_name, confusion_matrix), where classifier_name is a string 
    Aconfusion_matrix is a named tuple representing the two-by-two classification confusion matrix
    Domination is definded by smaller False positve and larger True positve
    """
    doms = []
    for color, matrix in classifiers:
      dominant = True
      for color_compare, matrix_compare in classifiers:
        if matrix.true_positive < matrix_compare.true_positive:
          if matrix.false_positive > matrix_compare.false_positive:
            dominant = False
            break
      if dominant:
        doms.append((color, matrix))
    return doms

Test 

['Blue', 'Red']

In [ ]:
# Example similar to the lecture notes

classifiers = [("Red", ConfusionMatrix(60, 40, 20, 80)), ("Green", ConfusionMatrix(40, 60, 30, 70)), ("Blue", ConfusionMatrix(80, 20, 50, 50)),]
print(sorted(label for (label, _) in roc_non_dominated(classifiers)))

['Blue', 'Red']


# Clustering


So far, every machine learning algorithm we have created takes a dataset which contains feature vectors, and the expected result. This is called supervised learning. But what if we don't have the expected result? Sometimes, we want to solve a problem that we do not have the answer to. 

So we turn to a class of algorithms that are unsupervised, meaning they are not given the correct answer, and instead have to work out what the answer is from the data alone. 

We will focus on a fundamental unsupervised algorithm called k-means clustering. It proceeds in two phases:

Assign each point in the training data to a class based on the "centroid" it is closest to.
Update the centroids to be the mean of the training data assigned to it.
The algorithm loops until step one repeats itself; that is, everything keeps the same class as the previous iteratio

## Q3

Update: see the notes added below.

Write a function k_means(dataset, centroids) that takes a dataset and k centroids and returns the tuple of updated centroids.

A dataset is a numpy array, where each row is a feature vector.

The centroids are passed in as a tuple of numpy arrays, where each array is a single centroid vector. Your returned tuple must be the same length as the input centroids tuple.

Note: In case of a tie (i.e., there are two closest centroids) choose the centroid that is first in the centroids tuple.

Added notes:
- Your algorithm needs to loop until convergence.
- In the second equation in the lecture notes, N_k is the number of data points that fall in cluster k.

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist

def k_means(dataset, centroids):
  """
  Takes a dataset and k centroids and returns the tuple of updated centroids.
  param dataset: numpy array, where each row is a feature vector.
  param k centroids: tuple of numpy arrays, where each array is a single centroid vector
  return: tuple must be the same length as the input centroids tuple.
  """
  np.warnings.filterwarnings('ignore', category=Warning)

  def centroid(data, centroids):
    deltas = cdist(data, centroids, 'euclidean')
    return np.array([np.argmin(delta) for delta in deltas])

  current = None
  while "Grass is green":
    #clusters = np.array([dataset[centroid(dataset, centroids) == cent] for cent in range(len(centroids))],dtype=object) # Removes warning but makes long floats???
    clusters = np.array([dataset[centroid(dataset, centroids) == cent] for cent in range(len(centroids))])
    new_cent = []
    for i in range(len(centroids)):
      if len(clusters[i]) == 0:
        new_cent.append(centroids[i])
      else:
        new_cent.append(clusters[i].mean(axis=0)) 
    centroids = tuple(new_cent)
    if np.array_equal(centroid(dataset, centroids), current):
      break
    current = centroid(dataset, centroids)

  return centroids

In [ ]:
dataset = np.array([
    [0.1, 0.1],
    [0.2, 0.2],
    [0.8, 0.8],
    [0.9, 0.9]
])
centroids = (np.array([0., 0.]), np.array([1., 1.]))
for c in k_means(dataset, centroids):
    print(c)

[0.15 0.15]
[0.85 0.85]


Test 1

[0.15 0.15]

[0.85 0.85]

In [ ]:
dataset = np.array([
    [0.1, 0.1],
    [0.2, 0.2],
    [0.8, 0.8],
    [0.9, 0.9]
])
centroids = (np.array([0., 1.]), np.array([1., 0.]))
for c in k_means(dataset, centroids):
    print(c)
# Think - what does this result 
#         mean for the partitions?

[0.5 0.5]
[1. 0.]


Test 2

[0.5 0.5]

[1. 0.]

In [ ]:
dataset = np.array([
    [0.1, 0.3],
    [0.4, 0.6],
    [0.1, 0.2],
    [0.2, 0.1]
])
centroids = (np.array([2., 5.]),)
for c in k_means(dataset, centroids):
    print(c)

[0.2 0.3]


Test 3
[0.2 0.3]

The obvious drawback with K-Means is the need to provide a set of starting centroids. The algorithm is very sensitive to its initialisation, so choosing "wrong" centroids to start with can produce wildly different results. So we get around this problem by guessing with many different starting centroids, then choosing the result that is best.

But how do we define "best"? In previous algorithms, we would compute some error against the true answers, but this is unsupervised - we don't know the answer. Instead, we have to describe what a good solution should look like, then let the algorithm maximise that. We define "good" as the separation of the clusters divided by the compactness of the clusters. 

Separation of a set of clusters is defined as the mean of the pairwise separation of clusters; the pairwise separation of clusters is smallest distance between points between the two clusters. That is, we want the mean of the smallest distances between clusters' points. Mathematically,

sep(C) = meanc, d ∈ C, c ≠ d min {dist(a, b) : a ∈ c, b ∈ d}

Compactness of a set of clusters is the mean of the compactness of each cluster; the compactness of a single cluster is its "diameter", meaning the largest distance between any two points in the cluster. That is, we want the mean of the largest distances within each cluster's points. Mathematically,

cpt(C) = meanc ∈ C max {dist(a, b) : a ∈ c, b ∈ c}

## Q4

Define a function k_means_random_restart(dataset, k, restarts, seed=None) where dataset is a numpy array with each row as a feature vector, k is the number of clusters, restarts is the number of times to try, and seed is the random number seed. This function should return the centroids as a numpy array that produce the best clustering, where "best" is separation divided by compactness.

Note: This question is unusual in that we have actually given you the function we want you to write. However, it has some dependencies we have not provided. You will need to supply the k_means, cluster_points and goodness helper functions yourself – although one of these should already be done!





In [ ]:
import hashlib
import numpy as np
from scipy.spatial.distance import cdist, pdist
#from itertools import combinations
import itertools

def k_means(dataset, centroids):
  """
  Takes a dataset and k centroids and returns the tuple of updated centroids.
  param dataset: numpy array, where each row is a feature vector.
  param k centroids: tuple of numpy arrays, where each array is a single centroid vector
  return: tuple must be the same length as the input centroids tuple.
  """
  np.warnings.filterwarnings('ignore', category=Warning)

  def centroid(data, centroids):
    deltas = cdist(data, centroids, 'euclidean')
    return np.array([np.argmin(delta) for delta in deltas])

  current = None
  while "Grass is green":
    #clusters = np.array([dataset[centroid(dataset, centroids) == cent] for cent in range(len(centroids))],dtype=object) # Removes warning but makes long floats???
    clusters = np.array([dataset[centroid(dataset, centroids) == cent] for cent in range(len(centroids))])
    new_cent = []
    for i in range(len(centroids)):
      if len(clusters[i]) == 0:
        new_cent.append(centroids[i])
      else:
        new_cent.append(clusters[i].mean(axis=0)) 
    centroids = tuple(new_cent)
    if np.array_equal(centroid(dataset, centroids), current):
      break
    current = centroid(dataset, centroids)

  return centroids

def pseudo_random(seed=0xdeadbeef):
    """generate an infinite stream of pseudo-random numbers"""
    state = (0xffffffff & seed)/0xffffffff
    while True:
        h = hashlib.sha256()
        h.update(bytes(str(state), encoding='utf8'))
        bits = int.from_bytes(h.digest()[-8:], 'big')
        state = bits >> 32
        r = (0xffffffff & bits)/0xffffffff
        yield r

def generate_random_vector(bounds, r):
    return np.array([(high - low) * next(r) + low for low, high in bounds])

####

def cluster_points(centroids, dataset):
  clusters = np.array([dataset[np.array([np.argmin(delta) 
  for delta in cdist(dataset, centroids, 'euclidean')]) == cent] 
  for cent in range(len(centroids))])
  return clusters

def separation(clusters):
  """sep(C) = meanc, d ∈ C, c ≠ d min {dist(a, b) : a ∈ c, b ∈ d}"""
  result = []
  for i, j in list(itertools.combinations(range(len(clusters)), 2)):
    result.append(np.amin(cdist(clusters[i], clusters[j], 'euclidean')))
  return np.mean(result)

def compactness(clusters):
  """cpt(C) = meanc ∈ C max {dist(a, b) : a ∈ c, b ∈ c}"""
  return np.mean([max(pdist(c, 'euclidean')) for c in clusters])

def goodness(clusters):
  """separation of the clusters divided by the compactness of the clusters"""
  return separation(clusters) / compactness(clusters)

####

def k_means_random_restart(dataset, k, restarts, seed=None):
  bounds = list(zip(np.min(dataset, axis=0), np.max(dataset, axis=0)))
  r = pseudo_random(seed=seed) if seed else pseudo_random()
  models = []
  for _ in range(restarts):
    random_centroids = tuple(generate_random_vector(bounds, r) for _ in range(k))
    new_centroids = k_means(dataset, random_centroids)
    clusters = cluster_points(new_centroids, dataset)
    if any(len(c) == 0 for c in clusters):
      continue
    models.append((goodness(clusters), new_centroids))
  return max(models, key=lambda x: x[0])[1]

In [ ]:
dataset = np.array([
    [0.1, 0.1],
    [0.2, 0.2],
    [0.8, 0.8],
    [0.9, 0.9]
])
centroids = k_means_random_restart(dataset, k=2, restarts=5)

for c in sorted([f"{x:8.3}" for x in centroid] for centroid in centroids):
    print(" ".join(c))

    0.15     0.15
    0.85     0.85


Test 1

    0.15     0.15
    
    0.85     0.85

In [ ]:
import sklearn.datasets
import sklearn.utils

iris = sklearn.datasets.load_iris()
data, target = sklearn.utils.shuffle(iris.data, iris.target, random_state=0)
train_data, train_target = data[:-5, :], target[:-5]
test_data, test_target = data[-5:, :], target[-5:]

centroids = k_means_random_restart(train_data, k=3, restarts=10)


# We suggest you check which centroid each 
# element in test_data is closest to, then see test_target.
# Note cluster 0 -> label 1
#      cluster 1 -> label 2
#      cluster 2 -> label 0

for c in sorted([f"{x:8.3}" for x in centroid] for centroid in centroids):
    print(" ".join(c))

     5.9     2.75      4.4     1.44
    5.02     3.44     1.46     0.25
    6.84     3.06     5.72     2.08


Test 2 

     5.9     2.75      4.4     1.44

    5.02     3.44     1.46     0.25
    
    6.84     3.06     5.72     2.08